In [1]:
import tensorflow as tf
from tensorflow import keras
import time, os

In [2]:
start = time.time() # 시작 시간 저장

In [3]:
# Detect hardware
try:
  tpu_resolver = tf.distribute.cluster_resolver.TPUClusterResolver() # TPU detection
except ValueError:
  tpu_resolver = None
#end of try:

if tpu_resolver:
  tf.config.experimental_connect_to_cluster(tpu_resolver)
  tf.tpu.experimental.initialize_tpu_system(tpu_resolver)
  strategy = tf.distribute.TPUStrategy(tpu_resolver)
  print('\n\nRunning on TPU ', tpu_resolver.cluster_spec().as_dict()['worker'])
else:
  strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
  print('\n\nRunning on CPU\n\n')
  
print("\n\nNumber of accelerators: ", strategy.num_replicas_in_sync,"\n\n")

INFO:tensorflow:Initializing the TPU system: grpc://10.79.155.98:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.79.155.98:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)




Running on TPU  ['10.79.155.98:8470']


Number of accelerators:  8 




In [4]:
def get_dataset():
    batch_size = 32
    num_val_samples = 10000

    # Return the MNIST dataset in the form of a `tf.data.Dataset`.
    (x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

    # Preprocess the data (these are Numpy arrays)
    x_train = x_train.reshape(-1, 784).astype("float32") / 255
    x_test = x_test.reshape(-1, 784).astype("float32") / 255
    y_train = y_train.astype("float32")
    y_test = y_test.astype("float32")

    # Reserve num_val_samples samples for validation
    x_val = x_train[-num_val_samples:]
    y_val = y_train[-num_val_samples:]
    x_train = x_train[:-num_val_samples]
    y_train = y_train[:-num_val_samples]
    return (
        tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch_size),
        tf.data.Dataset.from_tensor_slices((x_val, y_val)).batch(batch_size),
        tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size),
    )
#end of def:


In [5]:
def get_compiled_model():
    # Make a simple 2-layer densely-connected neural network.
    inputs = keras.Input(shape=(784,))
    x = keras.layers.Dense(256, activation="relu")(inputs)
    x = keras.layers.Dense(256, activation="relu")(x)
    outputs = keras.layers.Dense(10)(x)
    model = keras.Model(inputs, outputs)
    model.compile(
        optimizer=keras.optimizers.Adam(),
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[keras.metrics.SparseCategoricalAccuracy()],
    )
    return model
#end of def:

In [ ]:
# Create a MirroredStrategy.
print(tf.config.list_physical_devices("CPU"))
print(tf.config.list_physical_devices("GPU"))
gpus = tf.config.experimental.list_logical_devices("GPU")

if len(gpus) > 1:
  strategy = tf.distribute.MirroredStrategy([gpu.name for gpu in gpus])
  print('\n\nRunning on multiple GPUs ', [gpu.name for gpu in gpus])
elif len(gpus) == 1:
  strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
  print('\n\nRunning on single GPU ', gpus[0].name)
else:
  strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
  print("'\n\nRunning on CPU '")
#end of if
 
print("Number of devices: {}".format(strategy.num_replicas_in_sync))

# Open a strategy scope.
with strategy.scope():
    # Everything that creates variables should be under the strategy scope.
    # In general this is only model construction & `compile()`.
    model = get_compiled_model()

# Train the model on all available devices.
train_dataset, val_dataset, test_dataset = get_dataset()
model.fit(train_dataset, epochs=10, validation_data=val_dataset)

# Test the model on all available devices.
model.evaluate(test_dataset)

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
[]
'

Running on CPU '
Number of devices: 1
11493376/11490434 [==============================] - 0s 0us/step
Epoch 1/10
   1/1563 [..............................] - ETA: 1:49 - loss: 2.2881 - sparse_categorical_accuracy: 0.1875WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0008s vs `on_train_batch_end` time: 0.0097s). Check your callbacks.


1561/1563 [============================>.] - ETA: 0s - loss: 0.2259 - sparse_categorical_accuracy: 0.9327WARNING:tensorflow:Callbacks method `on_test_batch_end` is slow compared to the batch time (batch time: 0.0008s vs `on_test_batch_end` time: 0.0048s). Check your callbacks.


1563/1563 [==============================] - 16s 10ms/step - loss: 0.2257 - sparse_categorical_accuracy: 0.9328 - val_loss: 0.1235 - val_sparse_categorical_accuracy: 0.9635
Epoch 2/10
1563/1563 [==============================] - 16s 10ms/step - loss: 0.0951 - sparse_categorical_accuracy: 0.9712 - val_loss: 0.0971 - val_sparse_categorical_accuracy: 0.9703
Epoch 3/10
1563/1563 [==============================] - 16s 10ms/step - loss: 0.0608 - sparse_categorical_accuracy: 0.9814 - val_loss: 0.1069 - val_sparse_categorical_accuracy: 0.9692
Epoch 4/10
1563/1563 [==============================] - 17s 11ms/step - loss: 0.0413 - sparse_categorical_accuracy: 0.9871 - val_loss: 0.0966 - val_sparse_categorical_accuracy: 0.9740
Epoch 5/10
1563/1563 [==============================] - 16s 10ms/step - loss: 0.0354 - sparse_categorical_accuracy: 0.9886 - val_loss: 0.1072 - val_sparse_categorical_accuracy: 0.9733
Epoch 6/10
1563/1563 [==============================] - 16s 10ms/step - loss: 0.0276 - spar

In [ ]:
print("실행시간 :", round(time.time() - start, 2),"(초)")  # 현재시각 - 시작시간 = 실행 시간

CPU:

실행시간 : 63.14 (초)

TPU:

실행시간 : 218.56 (초)